In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt

## 准备工作

In [2]:
# 读取数据
weather = pd.read_csv('../Database/weatherstats_edmonton_hourly.csv')
weather.head()

,date_time_local,unixtime,pressure_station,pressure_sea,wind_dir,wind_dir_10s,wind_speed,wind_gust,relative_humidity,dew_point,temperature,windchill,humidex,visibility,health_index,cloud_cover_4,cloud_cover_8,cloud_cover_10,solar_radiation
0,2018-12-18 16:00:00 MST,1545174000,91.73,99.7,SW,23.0,5.0,NaN,64.0,-7.3,-1.3,-3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-12-18 15:00:00 MST,1545170400,91.67,99.7,S,18.0,5.0,NaN,65.0,-6.3,-0.5,-2.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN
2,2018-12-18 14:00:00 MST,1545166800,91.67,99.6,S,17.0,5.0,NaN,51.0,-6.5,2.7,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN
3,2018-12-18 13:00:00 MST,1545163200,91.66,99.6,SE,13.0,2.0,NaN,50.0,-6.4,2.9,NaN,NaN,NaN,3.9,NaN,NaN,NaN,NaN
4,2018-12-18 12:00:00 MST,1545159600,91.75,99.7,NaN,NaN,0.0,NaN,59.0,-6.6,0.5,NaN,NaN,NaN,4.1,NaN,NaN,NaN,NaN


In [61]:
# 读取数据
mosquito = pd.read_csv('../Database/Mosquito_Trap_Data.csv')
mosquito.head()

,Trap Date,Genus,Specific Epithet,Gender,IDd,Count,Trap Region,Include,Comparison Group,Latitude,Longitude,Location
0,09/15/2015 12:00:00 AM,Aedes,vexans,Female,NaN,3,Rural-West,NaN,Outer,53.562973,-113.787353,POINT (-113.787353 53.5629733)
1,05/12/1998 12:00:00 AM,Male,UnID,Male,UnID,1,Rural-North West,NaN,Peripheral,53.617221,-113.716182,POINT (-113.716182 53.617221)
2,08/28/2001 12:00:00 AM,Culex,tarsalis,Female,NaN,1,Rural-North West,NaN,Peripheral,53.617221,-113.716182,POINT (-113.716182 53.617221)
3,08/17/2004 12:00:00 AM,Culex,territans,Female,NaN,1,Rural-North West,NaN,Peripheral,53.617221,-113.716182,POINT (-113.716182 53.617221)
4,05/15/2018 12:00:00 AM,Culiseta,impatiens,Female,NaN,2,Rural-West,NaN,Outer,53.559020,-114.001190,POINT (-114.00119 53.559020000000004)


In [4]:
# 将日期中的 MST 删掉的方法
def format_date(series, offsets):
    date_formatted = []

    for date in series:
        date = date[0:offsets]
        date_formatted.append(date)

    date_formatted = pd.Series(date_formatted)
    return date_formatted

### 处理weather数据集

In [5]:
date_formatted_0=format_date(weather['date_time_local'],10)

In [6]:
# 判断格式化日期后的类型是否正确
type(date_formatted_0)

pandas.core.series.Series

In [7]:
# 改正格式后的日期重新赋值 -> weather dataset
weather['date_time_local'] = date_formatted_0
weather = weather.drop(columns=['unixtime'])
weather = weather.rename(columns={'date_time_local':'Date'})

In [8]:
weather.head()

,Date,pressure_station,pressure_sea,wind_dir,wind_dir_10s,wind_speed,wind_gust,relative_humidity,dew_point,temperature,windchill,humidex,visibility,health_index,cloud_cover_4,cloud_cover_8,cloud_cover_10,solar_radiation
0,2018-12-18,91.73,99.7,SW,23.0,5.0,NaN,64.0,-7.3,-1.3,-3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-12-18,91.67,99.7,S,18.0,5.0,NaN,65.0,-6.3,-0.5,-2.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN
2,2018-12-18,91.67,99.6,S,17.0,5.0,NaN,51.0,-6.5,2.7,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN
3,2018-12-18,91.66,99.6,SE,13.0,2.0,NaN,50.0,-6.4,2.9,NaN,NaN,NaN,3.9,NaN,NaN,NaN,NaN
4,2018-12-18,91.75,99.7,NaN,NaN,0.0,NaN,59.0,-6.6,0.5,NaN,NaN,NaN,4.1,NaN,NaN,NaN,NaN


In [9]:
# 按照时间进行数据聚合，取平均值
weather = weather.groupby('Date').agg('mean')

In [10]:
# 删除含有NaN的行和全部为NaN的列
weather = weather.dropna(axis=1,how='all')  # for column

In [11]:
weather.head()

,pressure_station,pressure_sea,wind_dir_10s,wind_speed,wind_gust,relative_humidity,dew_point,temperature,windchill,humidex,health_index
Date,,,,,,,,,,,
2016-12-18,92.502857,100.557143,27.571429,17.428571,33.666667,53.000000,-6.371429,2.271429,NaN,NaN,2.371429
2016-12-19,92.041667,99.987500,25.125000,15.625000,41.285714,52.250000,-8.091667,0.891667,-4.375000,NaN,2.737500
2016-12-20,92.529167,100.587500,23.208333,7.625000,NaN,59.625000,-10.787500,-3.945833,-7.041667,NaN,3.483333
2016-12-21,93.054583,101.150000,21.916667,11.625000,31.000000,59.625000,-7.987500,-0.808333,-6.750000,NaN,3.112500
2016-12-22,92.422917,100.433333,25.208333,7.250000,NaN,63.208333,-7.320833,-0.941667,-3.764706,NaN,3.783333


In [12]:
# 将index设置为第一列，为下面的resample做准备
weather = weather.reset_index()

# 统一时间周期，使用resample()方法，取平均值
weather['Date']=pd.DatetimeIndex(weather['Date'])
weather = weather.resample('W-TUE',on='Date').mean()

weather

,pressure_station,pressure_sea,wind_dir_10s,wind_speed,wind_gust,relative_humidity,dew_point,temperature,windchill,humidex,health_index
Date,,,,,,,,,,,
2016-12-20,92.357897,100.377381,25.301587,13.559524,37.476190,54.958333,-8.416865,-0.260913,-5.708333,NaN,2.864087
2016-12-27,93.168631,101.390476,19.511905,10.047619,31.000000,75.898810,-12.560119,-8.757738,-13.859244,NaN,2.829167
2017-01-03,93.679286,101.951786,25.904762,9.255952,30.125000,75.976190,-13.185119,-9.477976,-13.841165,NaN,2.861820
2017-01-10,93.957143,102.371429,21.922619,10.452381,31.750000,83.869048,-17.872619,-15.773214,-22.035714,NaN,2.700000
2017-01-17,93.060536,101.256548,21.851190,10.244048,43.846154,69.309524,-11.466667,-6.622024,-12.156004,NaN,3.556548
...,...,...,...,...,...,...,...,...,...,...,...
2018-11-20,93.802024,101.963631,21.857143,9.351190,33.333333,78.863095,-5.185119,-1.810714,-9.360317,NaN,2.430952
2018-11-27,92.844405,100.916071,17.178030,8.130952,31.000000,80.220238,-4.804167,-1.738095,-5.604365,NaN,2.520833
2018-12-04,93.199858,101.327640,21.318582,8.288043,31.357143,83.305383,-5.762060,-3.191046,-7.055100,NaN,2.491071


### 处理mosquito数据集

In [62]:
date_formatted_1 = format_date(mosquito['Trap Date'], 10)

In [63]:
type(date_formatted_1)

pandas.core.series.Series

In [64]:
mosquito['Trap Date'] = date_formatted_1

In [65]:
mosquito.head()

,Trap Date,Genus,Specific Epithet,Gender,IDd,Count,Trap Region,Include,Comparison Group,Latitude,Longitude,Location
0,09/15/2015,Aedes,vexans,Female,NaN,3,Rural-West,NaN,Outer,53.562973,-113.787353,POINT (-113.787353 53.5629733)
1,05/12/1998,Male,UnID,Male,UnID,1,Rural-North West,NaN,Peripheral,53.617221,-113.716182,POINT (-113.716182 53.617221)
2,08/28/2001,Culex,tarsalis,Female,NaN,1,Rural-North West,NaN,Peripheral,53.617221,-113.716182,POINT (-113.716182 53.617221)
3,08/17/2004,Culex,territans,Female,NaN,1,Rural-North West,NaN,Peripheral,53.617221,-113.716182,POINT (-113.716182 53.617221)
4,05/15/2018,Culiseta,impatiens,Female,NaN,2,Rural-West,NaN,Outer,53.559020,-114.001190,POINT (-114.00119 53.559020000000004)


In [66]:
mosquito = mosquito.rename(columns={'Trap Date':'Date'})
mosquito = pd.DataFrame(mosquito,columns=['Date','Gender','Count'])

In [67]:
mosquito.head()

,Date,Gender,Count
0,09/15/2015,Female,3
1,05/12/1998,Male,1
2,08/28/2001,Female,1
3,08/17/2004,Female,1
4,05/15/2018,Female,2


In [68]:
# 转换为统一的时间格式
import time
def format_date_2(series):
    date_formatted = []
    
    for date in series:
        date = time.strptime(date,"%m/%d/%Y")
        new_date = time.strftime("%Y-%m-%d",date)
        date_formatted.append(new_date)
        
    date_formatted = pd.Series(date_formatted)
    return date_formatted

In [69]:
# 转换后赋值
mosquito['Date'] = format_date_2(mosquito['Date'])

In [70]:
mosquito.head()

,Date,Gender,Count
0,2015-09-15,Female,3
1,1998-05-12,Male,1
2,2001-08-28,Female,1
3,2004-08-17,Female,1
4,2018-05-15,Female,2


In [71]:
# 分别对(雌雄)和(总数)进行groupby
# mosquito = mosquito.groupby('Date').agg('sum')
mosquito_with_gender = mosquito.groupby(['Gender','Date']).agg('sum')
mosquito_with_count = mosquito.groupby('Date').agg('sum')

In [72]:
mosquito_with_gender

Count
Gender Date             
Female 1990-05-22     34
       1990-05-29    838
       1990-06-05    341
       1990-06-12    223
       1990-06-19    737
...                  ...
Male   2020-08-25    336
       2020-09-01    138
       2020-09-08    259
       2020-09-15    298
       2020-09-22    258

[1005 rows x 1 columns]

In [73]:
mosquito_with_count

,Count
Date,
1990-05-22,34
1990-05-29,838
1990-06-05,341
1990-06-12,223
1990-06-19,737
...,...
2020-08-25,645
2020-09-01,195
2020-09-08,356


In [74]:
mosquito_with_gender = mosquito_with_gender.unstack().T
mosquito_with_gender

Gender            Female   Male
      Date                     
Count 1990-05-22    34.0    NaN
      1990-05-29   838.0    NaN
      1990-06-05   341.0    NaN
      1990-06-12   223.0    NaN
      1990-06-19   737.0    NaN
...                  ...    ...
      2020-08-25   309.0  336.0
      2020-09-01    57.0  138.0
      2020-09-08    97.0  259.0
      2020-09-15   112.0  298.0
      2020-09-22    72.0  258.0

[584 rows x 2 columns]

In [75]:
mosquito_with_gender = mosquito_with_gender.reset_index()
mosquito_with_gender

Gender,level_0,Date,Female,Male
0,Count,1990-05-22,34.0,NaN
1,Count,1990-05-29,838.0,NaN
2,Count,1990-06-05,341.0,NaN
3,Count,1990-06-12,223.0,NaN
4,Count,1990-06-19,737.0,NaN
...,...,...,...,...
579,Count,2020-08-25,309.0,336.0
580,Count,2020-09-01,57.0,138.0
581,Count,2020-09-08,97.0,259.0
582,Count,2020-09-15,112.0,298.0


In [76]:
mosquito_with_count = mosquito_with_count.reset_index()
mosquito_with_count

,Date,Count
0,1990-05-22,34
1,1990-05-29,838
2,1990-06-05,341
3,1990-06-12,223
4,1990-06-19,737
...,...,...
579,2020-08-25,645
580,2020-09-01,195
581,2020-09-08,356
582,2020-09-15,410


In [77]:
merge_mosquito = pd.merge(mosquito_with_gender,mosquito_with_count,how='inner',on='Date')
merge_mosquito = merge_mosquito.drop(columns=['level_0'])
merge_mosquito

,Date,Female,Male,Count
0,1990-05-22,34.0,NaN,34
1,1990-05-29,838.0,NaN,838
2,1990-06-05,341.0,NaN,341
3,1990-06-12,223.0,NaN,223
4,1990-06-19,737.0,NaN,737
...,...,...,...,...
579,2020-08-25,309.0,336.0,645
580,2020-09-01,57.0,138.0,195
581,2020-09-08,97.0,259.0,356
582,2020-09-15,112.0,298.0,410


In [78]:
# 用0替代NaN
merge_mosquito = merge_mosquito.fillna(0)

In [54]:
# 重命名，方便区分
merge_mosquito = merge_mosquito.rename(columns = {"Count": "Mos Count"})
print('merge_mosquito shape:',merge_mosquito.shape)

merge_mosquito shape: (584, 4)


In [79]:
merge_mosquito = merge_mosquito.set_index(merge_mosquito.columns[0])
merge_mosquito

,Female,Male,Count
Date,,,
1990-05-22,34.0,0.0,34
1990-05-29,838.0,0.0,838
1990-06-05,341.0,0.0,341
1990-06-12,223.0,0.0,223
1990-06-19,737.0,0.0,737
...,...,...,...
2020-08-25,309.0,336.0,645
2020-09-01,57.0,138.0,195
2020-09-08,97.0,259.0,356


### 合并数据

In [80]:
# 根据index对两个dataframe进行连接 -> 连接方式inner:只保留两者共有的部分
merge_result_based_date = weather.join(merge_mosquito,how='inner')

In [81]:
merge_result_based_date

,pressure_station,pressure_sea,wind_dir_10s,wind_speed,wind_gust,relative_humidity,dew_point,temperature,windchill,humidex,health_index,Female,Male,Count
Date,,,,,,,,,,,,,,
2017-05-09,93.290357,101.137619,23.577381,10.803571,36.642857,43.601190,1.245238,14.760119,NaN,26.125000,2.737500,26.0,0.0,26
2017-05-16,93.037976,100.916667,14.613095,14.202381,32.828571,65.797619,3.723214,10.885714,NaN,NaN,2.398214,23.0,6.0,29
2017-05-23,93.778452,101.660119,20.345238,10.875000,30.083333,45.613095,2.084524,15.286310,NaN,26.500000,2.463095,198.0,325.0,523
2017-05-30,93.317381,101.142262,20.892857,16.077381,34.324830,47.666667,3.257143,15.589881,NaN,25.000000,2.459524,150.0,213.0,363
2017-06-06,93.221905,101.009524,22.994048,13.238095,33.115584,54.500000,7.428571,17.765476,NaN,26.050000,2.852976,526.0,1130.0,1656
2017-06-13,92.799286,100.566667,18.101190,14.309524,31.658388,61.452381,8.191071,16.723214,NaN,25.864286,2.543333,1261.0,1708.0,2969
2017-06-20,93.194405,101.018452,23.720238,11.238095,35.162500,62.250000,7.318452,15.481548,NaN,NaN,2.275952,895.0,977.0,1872
2017-06-27,93.718393,101.567262,24.952381,18.089286,38.913483,53.613095,6.016667,16.300595,NaN,26.964286,2.364286,332.0,437.0,769
2017-07-04,93.421429,101.219643,23.494048,11.327381,32.795238,61.857143,10.094048,18.433333,NaN,25.851496,2.283333,1074.0,705.0,1779


In [82]:
merge_result_based_date.shape

(41, 14)

In [83]:
# 储存到文件
merge_result_based_date.to_csv('Weather_with_Mosquito_Weekly.csv')